<font size=7>Getting Started with SingleR</font>

This is my first notebook playin with single R. Lets see what I can do!

# How to Download Stuff

This section show commands that were used to download stuff

Command to download SingleR
``` python
devtools::install_github('dviraran/SingleR')
```

Remember to install umap with:
``` python
pip install umap-learn
```

# Set Up Notebook

First we load in Singler.

In [ ]:
library(SingleR)

## Supress Warnings

This stops SingleR from crash the notebook with too many warnings.
* [Reference Link](https://stackoverflow.com/questions/16194212/how-to-suppress-warnings-globally-in-an-r-script)

In [ ]:
options(warn=-1)
#options(warn=0) #<- to turn back on

## Load Packages

Then we load in packages found in the *"Description"* file

``` python
Imports: BiocManager,
    RColorBrewer,
    outliers,
    matrixStats,
    plyr,
    dplyr,
    ggplot2,
    pbmcapply,
    Matrix,
    GSEABase,
    GSVA,
    pheatmap,
    parallel,
    devtools,
    doFuture,
    data.table,
    foreach,
    singscore,
    doParallel
Suggests: Seurat (>= 2.2)
```

In [ ]:
library( BiocManager )
library( RColorBrewer)
library( outliers)
library( matrixStats)
library( plyr)
library( dplyr)
library( ggplot2)
library( pbmcapply)
library( Matrix)
library( GSEABase)
library( GSVA)
library( pheatmap)
library( parallel)
library( devtools)
library( doFuture)
library( data.table)
library( foreach)
library( singscore)
library(parallel)
library( doParallel)
library(dplyr)
library(Seurat)

## Load SingleR Files <small><font color="orange">Error in SingleR.References.R</font></small>

Files to Add:
``` python
HelperFunctions.R  SingleR.Create.R  SingleR.Object.R  SingleR.Plotting.R  SingleR.R  SingleR.References.R
```


In [ ]:
source("/home/jovyan/Public/SingleR/R/HelperFunctions.R")

In [ ]:
source("/home/jovyan/Public/SingleR/R/SingleR.Create.R")

In [ ]:
source("/home/jovyan/Public/SingleR/R/SingleR.Object.R")

In [ ]:
source("/home/jovyan/Public/SingleR/R/SingleR.Plotting.R")

In [ ]:
source("/home/jovyan/Public/SingleR/R/SingleR.R")

In [ ]:
source("/home/jovyan/Public/SingleR/R/SingleR.References.R")

# Load in CreateSinglerSeuratObject <font color="red">Trying to fix glitch</font>

In [ ]:
CreateSinglerSeuratObject = function(counts,annot=NULL,project.name,
                                     min.genes=200,technology='10X',
                                     species='Human',citation='',
                                     ref.list=list(),normalize.gene.length=F,
                                     variable.genes='de',fine.tune=T,
                                     reduce.file.size=T,do.signatures=F,
                                     min.cells=2,npca=10,regress.out='nUMI',
                                     do.main.types=T,reduce.seurat.object=T,
                                     temp.dir=NULL, numCores = SingleR.numCores) {
  if (!requireNamespace("Seurat", quietly = TRUE)) {
    stop("Seurat needed for this function to work. Please install it.",
         call. = FALSE)
  }
  
  print(project.name)
  print('Reading single-cell data...')
  sc.data = ReadSingleCellData(counts,annot)
  print('Create Seurat object...')
  seurat = SingleR.CreateSeurat(project.name,sc.data$counts,min.genes=
                                  min.genes,min.cells=min.cells,
                                regress.out=regress.out,npca=npca,
                                temp.dir=temp.dir)
  if (packageVersion('Seurat')>=3) {
    data = seurat@assays$RNA@data
    clusters = seurat@active.ident
  } else {
    data = seurat@data
    clusters = seurat@ident
    
  }
  
  orig.ident = sc.data$orig.ident[colnames(data)]
  counts = as.matrix(sc.data$counts[,colnames(data)])
  
  seurat@meta.data$orig.ident = factor(orig.ident)
  
  if(reduce.seurat.object==T) {
    if (packageVersion('Seurat')>=3) {
      seurat@assays$RNA@counts = matrix()
      seurat@assays$RNA@scale.data = matrix()
    } else {
      seurat@raw.data = c()
      seurat@scale.data = c()
      seurat@calc.params = list()
    }
  }
  
  print('Creat SingleR object...')
  
  singler = CreateSinglerObject(counts,orig.ident,project.name,
                                min.genes=min.genes,technology,species,
                                citation,ref.list,
                                normalize.gene.length,variable.genes,
                                fine.tune,do.signatures,
                                clusters,do.main.types,
                                reduce.file.size,temp.dir,numCores = numCores)
  
  singler$seurat = seurat 
  
  if (packageVersion('Seurat')>=3) {
    singler$meta.data$xy = seurat@reductions$tsne@cell.embeddings
    singler$meta.data$clusters = seurat@active.ident
    
  } else {
    singler$meta.data$xy = seurat@dr$tsne@cell.embeddings
    singler$meta.data$clusters = seurat@ident
  }
  singler
}

# Dissect `CreateSinglerSeuratObject`

The <font color="blue">SingleR</font> package is causing a bunch of errors. It's debug time! We are going to run the software the long way, line by line. 

We start by disecting the function `CreateSinglerSeuratObject` and running each of it's commands line by line.

In [ ]:
annot.file = 'GSE74923_annotations.txt'
counts.file =  'GSE74923_new.txt' 

The code for the `CreateSinglerSeuratObject` was copied from this location on [Github](https://github.com/dviraran/SingleR/blob/master/R/SingleR.Create.R)

## Set up initial variables

Below are the user inputted variables

In [ ]:
counts = 'GSE74923_new.txt' 
annot =  'GSE74923_new.txt'
project.name = 'GSE74923'

In [ ]:
variable.genes='de'; regress.out='nUMI' 
technology='C1'; species='Mouse'
citation='Kimmerling et al.'; reduce.file.size = F 
normalize.gene.length = T

We now add the default variables

In [ ]:

min.genes=200; 
ref.list=list()
variable.genes='de'; fine.tune=T
do.signatures=F
min.cells=2; npca=10
do.main.types=T; reduce.seurat.object=T
temp.dir=NULL

I added the following to prevent glitches

In [ ]:
 numCores =1

## Run "Reading single-cell data"

In [ ]:
  if (!requireNamespace("Seurat", quietly = TRUE)) {
    stop("Seurat needed for this function to work. Please install it.",
         call. = FALSE)
  }
  
  print(project.name)
  print('Reading single-cell data...')
  sc.data = ReadSingleCellData(counts,annot)

## Run "'Create Seurat object...'"

In [ ]:
  print('Create Seurat object...')
  seurat = SingleR.CreateSeurat(project.name,sc.data$counts,min.genes=
                                  min.genes,min.cells=min.cells,
                                regress.out=regress.out,npca=npca,
                                temp.dir=temp.dir)
  if (packageVersion('Seurat')>=3) {
    data = seurat@assays$RNA@data
    clusters = seurat@active.ident
  } else {
    data = seurat@data
    clusters = seurat@ident
    
  }
  
  orig.ident = sc.data$orig.ident[colnames(data)]
  counts = as.matrix(sc.data$counts[,colnames(data)])
  
  seurat@meta.data$orig.ident = factor(orig.ident)
  
  if(reduce.seurat.object==T) {
    if (packageVersion('Seurat')>=3) {
      seurat@assays$RNA@counts = matrix()
      seurat@assays$RNA@scale.data = matrix()
    } else {
      seurat@raw.data = c()
      seurat@scale.data = c()
      seurat@calc.params = list()
    }
  }

## Run "Creat SingleR object..." <font color="red">Error Here</font>

In [ ]:
print('Creat SingleR object...')  

``` python
singler = CreateSinglerObject(counts,orig.ident,project.name,
                            min.genes=min.genes,technology,species,
                            citation,ref.list,
                            normalize.gene.length,variable.genes,
                            fine.tune,do.signatures,
                            clusters,do.main.types,
                            reduce.file.size,temp.dir,numCores = numCores)
```

In [ ]:
singler = CreateSinglerObject(counts,orig.ident,project.name,
                            min.genes=min.genes,technology,species,
                            citation,ref.list,
                            normalize.gene.length,variable.genes,
                            fine.tune,do.signatures,
                            clusters,do.main.types,
                            reduce.file.size,temp.dir,numCores = numCores)

In [ ]:
  singler$seurat = seurat 
  
  if (packageVersion('Seurat')>=3) {
    singler$meta.data$xy = seurat@reductions$tsne@cell.embeddings
    singler$meta.data$clusters = seurat@active.ident
    
  } else {
    singler$meta.data$xy = seurat@dr$tsne@cell.embeddings
    singler$meta.data$clusters = seurat@ident
  }

### `Dissect` "CreateSinglerObject"

Error is happening in "CreateSinglerObject". Code is shown below:

``` python
CreateSinglerObject = function(counts,annot=NULL,project.name,
                               min.genes=0,technology='10X',
                               species='Human',citation='',
                               ref.list=list(),normalize.gene.length=F,
                               variable.genes='de',fine.tune=T,
                               do.signatures=F,clusters=NULL,
                               do.main.types=T,reduce.file.size=T,
                               temp.dir=NULL,numCores = SingleR.numCores) {
  
  sc.data = ReadSingleCellData(counts,annot)
  
  print(paste0('Dimensions of counts data: ',
               nrow(sc.data$counts),'x',ncol(sc.data$counts)))
  
  singler = list()
  
  
  N = colSums(counts>0)
  orig.ident = sc.data$orig.ident[N>=min.genes]
  counts = sc.data$counts[,N>=min.genes]
  
  if (normalize.gene.length == F) {
    sc.data.gl = counts
    rownames(sc.data.gl) = tolower(rownames(sc.data.gl))
  } else {
    if (species == 'Human') {
      sc.data.gl = TPM(counts,human_lengths)
    } else if (species == 'Mouse') {
      sc.data.gl = TPM(counts,mouse_lengths)
    }
  }
  
  if (length(ref.list)==0) {
    if (species == 'Mouse') {
      #if (!exists('immgen'))
      #  data('Immgen')
      #if (!exists('mouse.rnaseq'))
      #  data('Mouse-RNAseq')
      res = list(SingleR.CreateObject(sc.data.gl,immgen,clusters,species,
                                      citation,technology,
                                      do.main.types=do.main.types,
                                      variable.genes=variable.genes,
                                      fine.tune=fine.tune,numCores = numCores),
                 SingleR.CreateObject(sc.data.gl,mouse.rnaseq,clusters,
                                      species,citation,technology,
                                      do.main.types=do.main.types,
                                      variable.genes=variable.genes,
                                      fine.tune=fine.tune,numCores = numCores)
      )
    } else if (species == 'Human') {
      #if(!exists('hpca'))
      #  data ('HPCA')
      #if (!exists('blueprint_encode'))
      #  data('Blueprint_Encode')
      res = list(SingleR.CreateObject(sc.data.gl,hpca,clusters,species,
                                      citation,technology,
                                      do.main.types = do.main.types,
                                      variable.genes=variable.genes,
                                      fine.tune=fine.tune,numCores = numCores),
                 SingleR.CreateObject(sc.data.gl,blueprint_encode,
                                      clusters,species,citation,technology,
                                      do.main.types = do.main.types,
                                      variable.genes=variable.genes,
                                      fine.tune=fine.tune,numCores = numCores))
    }
  } else {
    res = lapply(ref.list, FUN=function(x) {
      SingleR.CreateObject(sc.data.gl,x,clusters,species,citation,technology,
                           do.main.types=do.main.types,
                           variable.genes=variable.genes,fine.tune=fine.tune,
                           numCores = numCores)
    })
  }
  
  singler$singler = res
  
  if (do.signatures==TRUE) {
    signatures = calculateSingScores(sc.data.gl,species=species)
    singler$signatures = signatures
    
  }
  
  if (species == 'Human') {
    kang = SingleR.CreateKangAnnotations(sc.data.gl)
    singler$other = kang$kang_annotation
  }
  
  singler$meta.data = list(project.name=project.name,orig.ident=orig.ident)
  
  if (reduce.file.size==T) {
    singler = remove.Unnecessary.Data.single(singler)
  }
  
  singler
  
}
```

## Return Singler

In [ ]:
singler

# Test SingleR  <font color="red">Errors</font>

In [ ]:
annot.file = 'GSE74923_annotations.txt'
counts.file =  'GSE74923_new.txt' 

In [ ]:
singler

Turned FineTuning off

In [ ]:
singler = CreateSinglerSeuratObject(counts.file, annot.file, 'GSE74923', min.genes=200,
                                    variable.genes='de', regress.out='nUMI', 
                                    technology='C1', species='Mouse',
                                    citation='Kimmerling et al.', reduce.file.size = F, 
                                    normalize.gene.length = T, numCores =1,
                                    fine.tune=F)
save(singler,file='GSE74923.RData')

In [ ]:
mclapply

In [ ]:
save(singler,file='GSE74923.RData')

In [ ]:
singler

In [ ]:
NotebookApp.iopub_msg_rate_limit

In [ ]:
singler

In [ ]:
#save(singler,file='GSE74923.RData'))

In [ ]:
#file.info(counts.file)

In [ ]:
CreateSeuratObject